# Data Generation

In [1]:
from datetime import datetime, timedelta
import json
import random

In [2]:
trades = ['framing carpenter','plumber', 'electrician','paver', 'dry wall','finishing carpenter', 'painter']
today = datetime.utcnow()

In [3]:
def number_of_rooms(max_rooms=6):
    # length = random.randrange(3, max_rooms, 1)
    # return range(0,length)
    return 6

In [4]:
def moc_deadlines(trades=[]):
    offset = random.randrange(0, 7, 1)
    start = datetime.utcnow() - timedelta(days=offset)
    return [
    (start + timedelta(weeks=index)).strftime('%Y-%m-%dT%H:%M:%SZ') 
     for index,trade in enumerate(trades)
    ]

In [5]:
def moc_room_pipeline(trades=[], deadlines=[]):
    return [
        {'tradesmen': trade, 'deadline': deadline} for trade, deadline in zip(trades,deadlines)
    ] 
        

In [6]:
def subset(trades=[]):
    start = random.randrange(0, int(len(trades)*0.75)+1, 1)
    return trades[start:]

In [7]:
def generate_building_data(floors=5):
    building_data = {}
    for floor in range(10):
        room_data = {}
        for room in number_of_rooms():
            trade_subset = subset(trades)
            pipeline = moc_room_pipeline(trade_subset, moc_deadlines(trade_subset))
            room_data[f'room_{floor}{room}'] = pipeline
        building_data[f'floor_{floor}'] = room_data
        
    return building_data

In [4]:
data = {}
buildings = [6,5,8]
for index, building in enumerate(buildings):
    print(f'floor_{index}')

floor_0
floor_1
floor_2


In [8]:
with open('buildingData.json', 'w') as json_file:
    json.dump(building_data, json_file)